# The CIFAR-10 dataset

CIFAR-10 is a very popular computer vision dataset provided by the Canadian Institute For Advanced Research (CIFAR). This dataset is used in many types of deep learning research for object recognition.

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class

The ‘10’ in this CIFAR-10 dataset refers to 10 classes. These 10 classes as shown in the above image are — Airplane, Automobile, Bird, Cat, Deer, Dog, Frog, Horse, Ship and Truck respectively.

The dataset is divided into five training batches and one test batch, each with 10000 images.

In [8]:
#Importing the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
import tensorflow as tf

from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### Load Dataset

In [2]:
# loading the dataset
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [4]:
# training dataset (images)
x_train.shape
x_test.shape

(10000, 32, 32, 3)

### Reshape data


*  The CNN model that is being used expects its input data to have this specific shape.


In [5]:
# Reshape the images to have a fixed size of (224, 224, 3)
y_train = y_train.reshape(-1, )
y_train
#y_classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

array([6, 9, 9, ..., 9, 1, 1], dtype=uint8)

### Exploring the dataset

In [ ]:
# showing an image that corresponds to the specific index
def showImages(x, y, index):
  plt.figure(figsize = (15,2))
  plt.imshow(x[index])
  #plt.xlabel(y_classes[y[index]])

showImages(x_train, y_train, 8)

### Normalizing and One-Hot-Encoding

In [11]:
# Normalizing it the range of [0,1] by diving it to 255 so that it can learn easy, better
# 255 because it is the highest pixel in an 8-bit image
x_train = x_train/255
x_test = x_test/255

# One-Hot-Encoding
# Converting the dataset into binary
# One-Hot-Encoding
y_train_en = to_categorical(y_train,10)
y_test_en = to_categorical(y_test,10)

In [ ]:
#y_train_en[0]

### CNN Model

In [12]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (4, 4), activation = "relu", input_shape = (32, 32, 3)))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(filters = 32, kernel_size = (4,4), activation="relu", input_shape = (32,32,3)))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(units = 128, activation = "relu"))
model.add(Dense(units = 10, activation = "softmax"))

model.compile(loss ="categorical_crossentropy", optimizer ="adam", metrics =["accuracy"])

## Train Model - Base model

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train_en, epochs = 17, verbose=1,validation_data=(x_test,y_test_en))

In [15]:
y_predictions = model.predict(x_test)

313/313 [==============================] - 1s 2ms/step


In [ ]:
y_predictions = [np.argmax(arr) for arr in y_predictions]

In [ ]:
print(classification_report(y_test, y_predictions))

### Model 1

In [25]:
model_1 = Sequential()
model_1.add(Conv2D(64,(4,4),input_shape = (32,32,3), activation ="relu"))
model_1.add(MaxPooling2D(pool_size = (2,2)))

model_1.add(Dropout(0.5))
model_1.add(Conv2D(64,(4,4),input_shape = (32,32,3), activation = "relu"))
model_1.add(MaxPooling2D(pool_size = (2,2)))

model_1.add(Dropout(0.25))
model_1.add(Flatten())
model_1.add(Dense(256,activation = "relu"))
model_1.add(Dense(10,activation = "softmax"))
model_1.compile(loss ="categorical_crossentropy", optimizer ="adam", metrics =["accuracy"])

In [ ]:
history = model_1.fit(x_train, y_train_en, epochs = 25, verbose=1,validation_data=(x_test,y_test_en))

In [ ]:
#y_predictions = model.predict(x_test)

### Model 2

In [ ]:
# Model_2 with more filters and learning rate
model_2 = Sequential()
model_2.add(Conv2D(64,(4,4),input_shape=(32,32,3),activation="relu"))
model_2.add(Conv2D(64,(4,4),input_shape=(32,32,3),activation="relu"))
model_2.add(MaxPooling2D(pool_size=(2,2)))
model_2.add(Dropout(0.4))
model_2.add(Conv2D(128,(4,4),input_shape=(32,32,3),activation="relu"))
model_2.add(Conv2D(128,(4,4),input_shape=(32,32,3),activation="relu"))
model_2.add(MaxPooling2D(pool_size=(2,2)))
model_2.add(Dropout(0.4))
model_2.add(Flatten())
model_2.add(Dense(1024, activation ="relu"))
model_2.add(Dense(1024, activation ="relu"))
model_2.add(Dense(10, activation =  "softmax"))
model_2.compile(loss ="categorical_crossentropy", optimizer =optimizers.Adam(learning_rate=0.0001), metrics =["accuracy"])
history = model_2.fit(x_train, y_train_en, epochs = 50, verbose=1,validation_data=(x_test,y_test_en))

In [ ]:
y_predictions = model_2.predict(x_test)

In [40]:
y_predictions = [np.argmax(arr) for arr in y_predictions]

In [ ]:
print(classification_report(y_test, y_predictions))

In [ ]:
y_true = label_binarize(y_test, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# Models
y_scores = model_2.predict(x_test)

n_classes = y_true.shape[1]

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_scores[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plotting ROC curves
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'Class {i} (area = {roc_auc[i]:0.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic: Model 2')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_predictions)

plt.figure(figsize = (14, 7))
sns.heatmap(cm, annot = True)
plt.ylabel("Prediction")
plt.xlabel("Actual")
plt.title("Confusion Matrix")
plt.show